# 数据预处理

In [7]:
import os
from PIL import Image
from array import *
from random import shuffle

# Load from and save to
Names = [['./train_to_ZJY','train'], ['./test_to_ZJY','test']]

for name in Names:
    
    data_image = array('B')
    data_label = array('B')

    FileList = []
    for dirname in os.listdir(name[0]): # 以列表形式返回train_to_ZJY下的所有文件{0,1]
        path = os.path.join(name[0],dirname) #用于寻找子文件夹
        for filename in os.listdir(path):
            if filename.endswith(".jpg"): #判断是否为jpg
                FileList.append(os.path.join(name[0],dirname,filename))

    shuffle(FileList) # 随机排序
    for filename in FileList:
        label = int(filename.split('\\')[1])
        Im = Image.open(filename)
        pixel = Im.load()
        width, height = Im.size
        for x in range(0,width):
            for y in range(0,height):
                data_image.append(pixel[y,x])
        data_label.append(label) # labels start (one unsigned byte each)
    hexval = "{0:#0{1}x}".format(len(FileList),6) # number of files in HEX
    header = array('B')
    header.extend([0,0,8,1,0,0])
    header.append(int('0x'+hexval[2:][:2],16))
    header.append(int('0x'+hexval[2:][2:],16))

    data_label = header + data_label

# additional header for images array

    if max([width,height]) <= 256:
        header.extend([0,0,0,width,0,0,0,height])
    else:
        raise ValueError('Image exceeds maximum size: 256x256 pixels');

    header[3] = 3 # Changing MSB for image data (0x00000803)

    data_image = header + data_image

    output_file = open(name[1]+'-images-idx3-ubyte', 'wb')
    data_image.tofile(output_file)
    output_file.close()

    output_file = open(name[1]+'-labels-idx1-ubyte', 'wb')
    data_label.tofile(output_file)
    output_file.close()

# gzip resulting files

for name in Names:
    os.system('gzip '+name[1]+'-images-idx3-ubyte') #二进制读取
    os.system('gzip '+name[1]+'-labels-idx1-ubyte')
# header for label array\

In [8]:
import struct
import numpy as np

In [9]:
labels_path = "train-labels-idx1-ubyte"
images_path = "train-images-idx3-ubyte"
with open(labels_path,"rb") as lbpath:
    magic,n  = struct.unpack(">II",lbpath.read(8))
    labels = np.fromfile(lbpath,dtype = np.uint8)
with open(images_path,"rb") as a:
    magic,num,rows,cols  = struct.unpack(">IIII",a.read(16))
    images = np.fromfile(a,dtype = np.uint8).reshape(len(labels),4096)

# 训练集和测试集的划分

* 将原始数据的测试集和训练集混合随机打乱后，选取80%为训练集，20%为测试集
* 总共选取23000张图片，测试集样本容量为18400，训练集样本容量为4600

In [10]:
labels.shape

(23000,)

In [11]:
images.shape

(23000, 4096)

In [12]:
list = []
for i in labels:
    if i == 1:
        list.append(1)
        continue
    list.append(-1)

In [13]:
training_images = images[:int(len(images)*0.8)]
test_images = images[int(len(images)*0.8):]
training_labels = list[:int(len(list)*0.8)]
test_labels = list[int(len(list)*0.8):]

In [14]:
print(training_images.shape)
print(test_images.shape)
print(len(training_labels))
print(len(test_labels))

(18400, 4096)
(4600, 4096)
18400
4600


#  perceptron

## 基本定义

![jupyter](./perceptron.png)

* 线性方程: $$w ⋅ x + b = 0$$
* 对于特征空间中的一个超平面 S ，其中 w是超平面的法向量， b是超平面的截距。这个超平面将特征空间划分为两个部分。位于两部分的点（特征向量）分别被分为正、负两类。因此超平面S称为分离超平面，如上图所示；
* 感知机通过训练训练集数据求得感知机模型，即求得模型参数 w， b。通过学习得到的感知机模型，对于新的输入实例给出其对应的输出类别；

## 学习策略

为求最优w,b，加入感知机学习的损失函数，考虑误分类点到超平面S的总距离，那么会出现两个问题，怎么判断误分类点？怎么计算这些点到超平面S的总距离？

误分类点判断


* 对于一个误分类点 $(x_{i},y_{i})$而说，当$ w ⋅ x_{i} + b > 0$时，$y_{i} = -1$，当 $w ⋅ x_{i} + b < 0$ 时，$y_{i} = 1$,所以有$$ -y_{i} ⋅( w ⋅ x_{i} + b ) > 0 $$


损失函数

* 误分类点到超平面的总距离(假设有m个误分类点): $$   - \frac{1 }{\mid\mid w \mid\mid}⋅ \sum_{j=0}^{m}(y_{j}⋅(w ⋅ x_{j} + b)) $$
* 忽略$ \frac{1 }{\mid\mid w \mid\mid}$,得到损失函数: $$ LOSS = - \sum_{j=0}^{m}(y_{j}⋅(w ⋅ x_{j} + b)) $$

w,b的计算

采用梯度下降算法：分别对w和y求导得：$$  - \nabla_{w}L(w,b) = - \sum_{j=0}^{m}(y_{j}x_{j}) $$ $$  - \nabla_{b}L(w,b) = - \sum_{j=0}^{m}(y_{j}) $$ 

故对误分类点w,b的更新如下： $$ w = w +\eta (y_{j}x_{j})$$ $$ b = b +\eta (y_{j})$$

重复执行上述步骤直到一次更新后不再有误分类点。而且由于初值和误分类点的选择顺序不同，最终结果可以有无穷多个。其实这很容易理解，考虑平面上线性可分的数据集，当然存在无穷多个超平面可以将其分开。另外可以证明误分类的次数是有上届的，经过有限次搜索可以找到将训练数据集完全分开的超平面，也就是说，当训练数据集线性可分时，感知机学习算法原始形式迭代是收敛的。

## 代码实现

In [15]:
import random

下面算法中遍历所有的点，出现误分类点，就更新一次w和b，其中loop_max为迭代次数，即重复以上步骤的次数

In [16]:
# 感知机学习算法
def perceptron(image,label,eta,loop_max): #images,labels,learning rate,迭代次数
    eta=0.5 # 学习率
    features=image.shape[1]   # x特征列数量
    w=np.array([x*0 for x in range(0,features)])
    b=0
    for times in range(loop_max): #迭代次数
        for i in range(len(image)):
            x = image[i]
            y = label[i]
            if y*(w@x+b)<=0: # @符号作用同np.dot
                w=w+eta*y*x
                b=b+eta*y
    return w,b

* 迭代1000的w,b值

In [25]:
w,b = perceptron(image = training_images,label = training_labels,eta = 0.5,loop_max = 1000) 

In [26]:
# 符号函数(类似激活函数，得到预测结果)
def sign(v):
    if v > 0:
        return 1
    else:
        return -1

In [27]:
#预测函数
def predict(image,label):
    fault = 0
    predict_result = []
    for x in image:
        predict_result.append(sign(w@x+b))
    for i in range(len(predict_result)):
        if  predict_result[i]!=label[i]:
            fault+=1
    accuracy = 1-(fault/len(label)) 
    return accuracy

In [28]:
predict(image=test_images,label = test_labels)

0.5106521739130434

In [30]:
predict(image=training_images,label = training_labels)

0.6336413043478261